# PSyIR Example 2 - Semantic Navigation

This example shows how to traverse the PSyIR using semantic navigation. Again we begin with Fortran conforming to the NEMO coding standards:

In [ ]:
code = '''program test
  implicit none
  logical, parameter :: ln_do_this = .TRUE.
  integer, parameter :: jpk=10
  real, dimension(jpk) :: b
  integer :: ji,jj,jk
  if(ln_do_this)then
    do jk=1,jpk
      b(jk) = 0.0
    end do
  else
    b(1) = -1.0
  end if
end program test'''

Next, we use PSyclone to create the PSyclone Internal Representation of this code:

In [ ]:
from fparser.common.readfortran import FortranStringReader
reader = FortranStringReader(code)
from fparser.two.parser import ParserFactory
parser = ParserFactory().create(std="f2003")
parse_tree = parser(reader)

from psyclone.psyGen import PSyFactory
psy = PSyFactory("nemo").create(parse_tree)

invoke = psy.invokes.invoke_list[0]
schedule = invoke.schedule

schedule.view()

Previously, we learned that each PSyIR node has the `parent` and `children` attributes. However, where appropriate, certain classes of node also support semantic navigation. The `If` node for instance has the logical expression as its first child, the body as its second and the body of any `else` clause as its third.  Similarly, the `Loop` node has its loop bounds and increment as its first three children and the loop body as the fourth. Therefore, in order to simplify navigation and insulate it from any future changes in tree structure, the `If` and `Loop` nodes have the `xx_body` methods:

In [ ]:
if_block = schedule.children[0]
body = if_block.if_body
print("If body: ", type(body.children[0]))
loop = body.children[0]
loop_body = loop.loop_body
print("Loop body: ", type(loop_body.children[0]))

In [ ]:
else_body = if_block.else_body
print("Else body: ", type(else_body.children[0]))

The logical condition associated with the `If` is accessed using the `condition` property:

In [ ]:
print("If condition: ", if_block.condition)

Finally, the `Assignment` node has the `lhs` and `rhs` methods:

In [ ]:
assign = else_body.children[0]
print("LHS = ", assign.lhs.name)
print("RHS = ", type(assign.rhs))

Please see the [User Guide](https://psyclone.readthedocs.io/en/stable/psyir.html#psyir-the-psyclone-internal-representation) for more details on navigating the PSyIR.

Congratulations, you have finished the PSyIR section of the tutorial.